## Purpose:

According to and article from [brookfieldresidential.com](https://stories.brookfieldresidential.com/homebuyersschool/duplex-vs.-single-family-home-whats-the-difference-and-which-one-should-i-invest-in), duplexes might be more highly valued than single family homes. Using the Seattle area real estate information, I'll run some analysis to determine the accuracy of this claim.

In [1]:
#add auto reload for src function testing
%load_ext autoreload
%autoreload 2

#let's add the project directory to our module path
import os
import sys

module_path = os.path.abspath(os.path.join(os.pardir, os.pardir))
if module_path not in sys.path:
    sys.path.append(module_path)
    
    
#also import all of our modules
import pandas as pd
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns
from src import data_cleaning

#and here is our data directiory
data_folder = '../../data/'

Let's open our data and see what it looks like. I'm looking for both housing prices and whether or not they are duplexes.

## Optional

The following lines of code will take a while to load. This is because the csv files are very large and contain data that we don't need. We solved this by creating new csv files that contain data for only 2019. Read through the following section to see how we did that, and uncomment the code if you want to follow along. Otherwise skip ahead to 2019 data import header.

### EXTR_RPSale.csv

It looks like this data set is what contains the sale price as well as some interesting characteristics of the property like whether it's historic or not. 

In [3]:
# rp_sale = pd.read_csv(data_folder+'EXTR_RPSale.csv')
# rp_sale.info()
# rp_sale.head()

### Date
I know I only want to look at records from 2019 so I'm going to filter out the data to only include entries from that date.

First I'll convert document date to datetime:

In [4]:
# rp_sale['DocumentDate'] = pd.to_datetime(rp_sale['DocumentDate'])

# rp_sale.info()

Now I'm going to create a function so that I can make a dataframe mask using apply. It will check whether the year attribute of a datetime object equals 2019 and, if so, it will return True, otherwise False.

In [5]:
# def in_2019(dateTime):
#     if dateTime.year == 2019:
#         return True
#     else:
#         return False

In [6]:
# mask_2019 = rp_sale['DocumentDate'].apply(in_2019)


# rp_sale_2019 = rp_sale[mask_2019]
# rp_sale_2019.info()
# rp_sale_2019.head(20)

I want to use this dataframe for all my analysis, so I'm going to export it as a csv, and I'll create a function to automatically do this as well.

In [7]:
#uncomment line below to create file

#rp_sale_2019.to_csv(data_folder+'EXTR_RPSale_2019.csv')

## Function testing:

In [8]:
# rp_sale_2019 = data_cleaning.filter_data_by_year(rp_sale)

In [9]:
# rp_sale_2019.info()

In [10]:
# data_cleaning.create_2019_sale_csv(rp_sale)

# 2019 data import

In [14]:
rp_sale = pd.read_csv(data_folder+'EXTR_RPSale_2019.csv')

In [15]:
rp_sale.head()

,Unnamed: 0,ExciseTaxNbr,Major,Minor,DocumentDate,SalePrice,RecordingNbr,Volume,Page,PlatNbr,...,PropertyType,PrincipalUse,SaleInstrument,AFForestLand,AFCurrentUseLand,AFNonProfitUse,AFHistoricProperty,SaleReason,PropertyClass,SaleWarning
0,72,2999169,919715,200,2019-07-08,192000,20190712001080,,,,...,3,2,3,N,N,N,N,1,3,
1,236,3000673,894444,200,2019-06-26,185000,20190722001395,,,,...,3,2,3,N,N,N,N,1,3,
2,257,3027422,213043,120,2019-12-20,560000,20191226000848,,,,...,11,6,3,N,N,N,N,1,8,
3,302,3002257,940652,630,2019-07-22,435000,20190730001339,,,,...,11,6,3,N,N,N,N,1,8,
4,446,3018109,152504,9008,2019-10-18,7600000,20191030001615,,,,...,3,7,3,N,N,N,N,1,2,


## EXTR_ResBldg.csv

I might need to do the same thing for res_bldg as well, so let's check that one out too.

In [13]:
res_bldg.info()
res_bldg.head()

NameError: name 'res_bldg' is not defined

It looks like it's just information about the building itself, so I'm not worried about filtering this by date.